# MLOps Manual to Repeatable Workflow

## Contents

- [Introduction](#Introduction)
- [Training pipeline with SageMaker Pipelines](#Training-pipeline-with-SageMaker-Pipelines)
    - [Pipeline inputs](#Pipeline-inputs)
    - [SageMaker Processing step](#SageMaker-Processing-step)
    - [SageMaker Training step](#SageMaker-Training-step)
    - [Model evaluation step](#Model-evaluation-step)
    - [Register model in Model Registry step](#Register-model-in-Model-Registry-step)
    - [Assemble the training pipeline](#Assemble-the-training-pipeline)
    - [Execute the training pipeline](#Execute-the-training-pipeline)
- [Deployment pipeline with SageMaker Pipelines](#Deployment-pipeline-with-SageMaker-Pipelines)
    - [Assemble the deployment pipeline](#Assemble-the-deployment-pipeline)
    - [Execute the deployment pipeline](#Execute-the-deployment-pipeline)
    - [Test the SageMaker endpoint](#Test-the-SageMaker-endpoint)

## Introduction

In this Notebook which will explore the orchestration stage of ML workflow. Please see the diagram below for the various maturity stages of MLOps

Here, we will put on the hat of a `DevOps/MLOps Engineer` and perform the task of orchestration which includes building pipeline steps that include all the various ML Workflows components into one singular entity. This pipeline entity accomplishes a repeatable and reliable orchestration of each step in the ML workflow.

![](./images/mlops-maturity.png)

For this task we will be using Amazon SageMaker Pipeline capabilities.

Let's get started!

**Imports**

In [ ]:
# Processing imports
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput, ScriptProcessor

# SageMaker Pipeline imports
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import ProcessingStep, TrainingStep, CreateModelStep, TransformStep
from sagemaker.workflow.model_step import ModelStep

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

# Other imports
import json
from time import gmtime, strftime
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.model import Model
from sagemaker.tuner import IntegerParameter, HyperparameterTuner
from sagemaker.inputs import TrainingInput
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)

# To test the endpoint once it's deployed
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer, CSVDeserializer
from sagemaker.workflow.pipeline_context import PipelineSession
import sagemaker
import json
import boto3
from sagemaker.model_metrics import ModelMetrics, MetricsSource
import pandas as pd
from sagemaker.feature_store.feature_group import FeatureGroup

In [ ]:
# Useful SageMaker variables
session = PipelineSession()
bucket = session.default_bucket()
role_arn= sagemaker.get_execution_role()
region = session.boto_region_name
sagemaker_client = boto3.client('sagemaker')
aws_account_id = boto3.client('sts').get_caller_identity().get('Account')
print(role_arn)

In [ ]:
# Some useful variable
output_path=f's3://{bucket}/mlops-workshop/data/sm_processed'  
model_package_group_name = 'synthetic-housing-models'


## Upload Raw Data set
Upload the Raw data set for processing 

In [ ]:
# Upload raw data to S3
raw_data_s3_prefix = 'mlops_workshop/data/raw'
raw_s3 = session.upload_data(path='./data/raw/house_pricing.csv', key_prefix=raw_data_s3_prefix)

**Session variables**

In [ ]:
def create_lambda_iam_role(role_name):
    iam = boto3.client("iam")
    try:
        response = iam.create_role(
            RoleName = role_name,
            AssumeRolePolicyDocument = json.dumps({
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": "lambda.amazonaws.com"
                        },
                        "Action": "sts:AssumeRole"
                    }
                ]
            }),
            Description='Role for Lambda to call SageMaker'
        )

        role_arn = response['Role']['Arn']

        response = iam.attach_role_policy(
            RoleName=role_name,
            PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
        )

        response = iam.attach_role_policy(
            PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess',
            RoleName=role_name
        )

        return role_arn

    except iam.exceptions.EntityAlreadyExistsException:
        print(f'Using ARN from existing role: {role_name}')
        response = iam.get_role(RoleName=role_name)
        print("Done")
        return response['Role']['Arn']
    try:
        response = iam.create_role(
            RoleName = role_name,
            AssumeRolePolicyDocument = json.dumps({
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": "lambda.amazonaws.com"
                        },
                        "Action": "sts:AssumeRole"
                    }
                ]
            }),
            Description='Role for Lambda to call SageMaker'
        )

        role_arn = response['Role']['Arn']

        response = iam.attach_role_policy(
            RoleName=role_name,
            PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
        )

        response = iam.attach_role_policy(
            PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess',
            RoleName=role_name
        )
        print("Done")

        return role_arn

    except iam.exceptions.EntityAlreadyExistsException:
        print(f'Using ARN from existing role: {role_name}')
        response = iam.get_role(RoleName=role_name)
        print("Done")
        return response['Role']['Arn']

In [ ]:
lambda_role = create_lambda_iam_role('LambdaSageMakerExecutionRole')
print(lambda_role)
print(raw_s3)

## Training pipeline with SageMaker Pipelines

An Amazon [SageMaker Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) pipeline is a series of interconnected steps that is defined by a JSON pipeline definition. This pipeline definition encodes a pipeline using a directed acyclic graph (DAG). This DAG gives information on the requirements for and relationships between each step of your pipeline. The structure of a pipeline's DAG is determined by the data dependencies between steps. These data dependencies are created when the properties of a step's output are passed as the input to another step. The following image is a pipeline DAG that we'll be creating for our training pipeline:

![](./images/sagemaker-pipelines-dag.png)

#### Pipeline inputs

You can give a pipeline inputs to make it reusable (you'll be able to override these inputs upon executing the pipeline later in the notebook).

In [ ]:
processing_instance_count = ParameterInteger(
    name='ProcessingInstanceCount',
    default_value=1
)

processing_instance_type = ParameterString(
    name='ProcessingInstanceType',
    default_value='ml.m5.xlarge'
)

#### SageMaker Processing step

This should look very similar to the SageMaker Training job you did in notebook 2. The only new line of code is the `ProcessingStep` line at the bottom of the cell below.

In [ ]:
!mkdir -p pipeline_scripts

In [ ]:
%%writefile ./pipeline_scripts/preprocessing.py
import pandas as pd
import argparse
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


def read_parameters():
    """
    Read job parameters
    Returns:
        (Namespace): read parameters
    """
    parser = argparse.ArgumentParser()
    parser.add_argument('--train_size', type=float, default=0.6)
    parser.add_argument('--val_size', type=float, default=0.2)
    parser.add_argument('--test_size', type=float, default=0.2)
    parser.add_argument('--random_state', type=int, default=42)
    parser.add_argument('--target_col', type=str, default='PRICE')
    params, _ = parser.parse_known_args()
    return params


def change_target_to_first_col(df, target_col):
    # shift column 'PRICE' to first position
    first_column = df.pop(target_col)
  
    # insert column using insert(position,column_name,
    # first_column) function
    df.insert(0, target_col, first_column)
    return df


def split_dataset(df, train_size, val_size, test_size, random_state=None):
    """
    Split dataset into train, validation and test samples
    Args:
        df (pandas.DataFrame): input data
        train_size (float): ratio of data to use as training dataset
        val_size (float): ratio of data to use as validation dataset
        test_size (float): ratio of data to use as test dataset
        random_state (int): Pass an int for reproducible output across multiple function calls.
    Returns:
        df_train (pandas.DataFrame): train dataset
        df_val (pandas.DataFrame): validation dataset
        df_test (pandas.DataFrame): test dataset
    """
    if (train_size + val_size + test_size) != 1.0:
        raise ValueError("train_size, val_size and test_size must sum up to 1.0")
    rest_size = 1 - train_size
    df_train, df_rest = train_test_split(
        df,
        test_size=rest_size,
        train_size=train_size,
        random_state=random_state
    )
    df_val, df_test = train_test_split(
        df_rest,
        test_size=(test_size / rest_size),
        train_size=(val_size / rest_size),
        random_state=random_state
    )
    df_train.reset_index(inplace=True, drop=True)
    df_val.reset_index(inplace=True, drop=True)
    df_test.reset_index(inplace=True, drop=True)
    train_perc = int((len(df_train)/len(df)) * 100)
    print(f"Training size: {len(df_train)} - {train_perc}% of total")
    val_perc = int((len(df_val)/len(df)) * 100)
    print(f"Val size: {len(df_val)} - {val_perc}% of total")
    test_perc = int((len(df_test)/len(df)) * 100)
    print(f"Test size: {len(df_test)} - {test_perc}% of total")
    return df_train, df_val, df_test


def scale_dataset(df_train, df_val, df_test, target_col):
    """
    Fit StandardScaler to df_train and apply it to df_val and df_test
    Args:
        df_train (pandas.DataFrame): train dataset
        df_val (pandas.DataFrame): validation dataset
        df_test (pandas.DataFrame): test dataset
        target_col (str): target col
    Returns:
        df_train_transformed (pandas.DataFrame): train data scaled
        df_val_transformed (pandas.DataFrame): val data scaled
        df_test_transformed (pandas.DataFrame): test data scaled
    """
    scaler_data = StandardScaler()
    
    # fit scaler to training dataset
    print("Fitting scaling to training data and transforming dataset...")
    df_train_transformed = pd.DataFrame(
        scaler_data.fit_transform(df_train), 
        columns=df_train.columns
    )
    df_train_transformed[target_col] = df_train[target_col]
    
    # apply scaler to validation and test datasets
    print("Transforming validation and test datasets...")
    df_val_transformed = pd.DataFrame(
        scaler_data.transform(df_val), 
        columns=df_val.columns
    )
    df_val_transformed[target_col] = df_val[target_col]
    df_test_transformed = pd.DataFrame(
        scaler_data.transform(df_test), 
        columns=df_test.columns
    )
    df_test_transformed[target_col] = df_test[target_col]
    return df_train_transformed, df_val_transformed, df_test_transformed


print(f"===========================================================")
print(f"Starting pre-processing")
print(f"Reading parameters")

# reading job parameters
args = read_parameters()
print(f"Parameters read: {args}")

# set input and output paths
input_data_path = "/opt/ml/processing/input/house_pricing.csv"
train_data_path = "/opt/ml/processing/output/train"
val_data_path = "/opt/ml/processing/output/validation"
test_data_path = "/opt/ml/processing/output/test"

try:
    os.makedirs(train_data_path)
    os.makedirs(val_data_path)
    os.makedirs(test_data_path)
except:
    pass


# read data input
df = pd.read_csv(input_data_path)

# move target to first col
df = change_target_to_first_col(df, args.target_col)

# split dataset into train, validation and test
df_train, df_val, df_test = split_dataset(
    df,
    train_size=args.train_size,
    val_size=args.val_size,
    test_size=args.test_size,
    random_state=args.random_state
)

# scale datasets
df_train_transformed, df_val_transformed, df_test_transformed = scale_dataset(
    df_train, 
    df_val, 
    df_test,
    args.target_col
)

print("Saving data")
df_train_transformed.to_csv(train_data_path+'/train.csv', sep=',', index=False, header=False)
df_val_transformed.to_csv(val_data_path+'/validation.csv', sep=',', index=False, header=False)
df_test_transformed.to_csv(test_data_path+'/test.csv', sep=',', index=False, header=False)



print(f"Ending pre-processing")
print(f"===========================================================")

In [ ]:
preprocess_data_processor = SKLearnProcessor(
    framework_version='0.23-1',
    role=role_arn,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name='preprocess-data',
    sagemaker_session=session,
)

preprocess_dataset_step = ProcessingStep(
    name='PreprocessData',
    code='./pipeline_scripts/preprocessing.py',
    processor=preprocess_data_processor,
    inputs=[
        ProcessingInput(
            source=raw_s3,
            destination='/opt/ml/processing/input',
            s3_data_distribution_type='ShardedByS3Key'
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name='train',
            destination=f'{output_path}/train',
            source='/opt/ml/processing/train'
        ),
        ProcessingOutput(
            output_name='validation',
            destination=f'{output_path}/validation',
            source='/opt/ml/processing/validation'
        ),
        ProcessingOutput(
            output_name='test',
            destination=f'{output_path}/test',
            source='/opt/ml/processing/test'
        )
    ]
)

#### SageMaker Training step

This should look very similar to the SageMaker Training job you did in notebook 2. The only new line of code is the `TrainingStep` line at the bottom of the cell below.

In [ ]:
# Tuned hyperparameters
hyperparameters = {
    "max_depth": "7",
    "gamma": "2",
    "alpha": "375",
    "objective": "reg:squarederror",
    "num_round": "50",
    "verbosity": "2",
    "eval_metric": "mse"
}

train_instance_type = 'ml.c5.xlarge'


# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
#xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.5-1")
xgboost_container = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type="ml.m5.xlarge",
)
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(
    image_uri=xgboost_container, 
    hyperparameters=hyperparameters,
    role=role_arn,
    instance_count=1, 
    instance_type='ml.m5.2xlarge', 
    volume_size=5, # 5 GB 
    sagemaker_session=session
)

training_step = TrainingStep(
    name='TrainModel',
    estimator=estimator,
    inputs={
        'train': TrainingInput(
            s3_data=preprocess_dataset_step.properties.ProcessingOutputConfig.Outputs[
                'train'
            ].S3Output.S3Uri,
            content_type='text/csv'
        ),
        'validation': TrainingInput(
            s3_data=preprocess_dataset_step.properties.ProcessingOutputConfig.Outputs[
                'validation'
            ].S3Output.S3Uri,
            content_type='text/csv'
        )
    },
)

#### Model evaluation step

After the training step in our pipeline, we'll want to then evaluate our model's performance. To do that, we can create a SageMaker Processing Step and pass in some code to do the model evaluation.

In [ ]:
%%writefile ./pipeline_scripts/evaluation.py
import json
import pathlib
import pickle
import tarfile

import joblib
import numpy as np
import pandas as pd
import xgboost

from math import sqrt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


if __name__ == "__main__":
    model_path = f"/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path) as tar:
        tar.extractall(path=".")

    model = pickle.load(open("xgboost-model", "rb"))

    test_path = "/opt/ml/processing/test/test.csv"
    df = pd.read_csv(test_path, header=None)

    y_test = df.iloc[:, 0].to_numpy()
    df.drop(df.columns[0], axis=1, inplace=True)

    X_test = xgboost.DMatrix(df.values)

    predictions = model.predict(X_test)

    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = sqrt(mse)
    r2 = r2_score(y_test, predictions)
    std = np.std(y_test - predictions)
    report_dict = {
        'regression_metrics': {
            'mae': {
                'value': mae,
                'standard_deviation': std,
            },
            'mse': {
                'value': mse,
                'standard_deviation': std,
            },
            'rmse': {
                'value': rmse,
                'standard_deviation': std,
            },
            'r2': {
                'value': r2,
                'standard_deviation': std,
            },
        },
    }

    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))

In [ ]:
evaluation_processor = ScriptProcessor(
    image_uri=xgboost_container,
    command=["python3"],
    role=role_arn,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name='evaluation',
    sagemaker_session=session,
)

In [ ]:
# Specify where we'll store the model evaluation results so
# that other steps can access those results
evaluation_report = PropertyFile(
    name='EvaluationReport',
    output_name='evaluation',
    path='evaluation.json',
)

evaluation_step = ProcessingStep(
    name='EvaluateModel',
    processor=evaluation_processor,
    inputs=[
        ProcessingInput(
            source=training_step.properties.ModelArtifacts.S3ModelArtifacts,
            destination='/opt/ml/processing/model',
        ),
        ProcessingInput(
            source=preprocess_dataset_step.properties.ProcessingOutputConfig.Outputs['test'].S3Output.S3Uri,
            destination='/opt/ml/processing/test',
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name='evaluation', source='/opt/ml/processing/evaluation'
        ),
    ],
    code='./pipeline_scripts/evaluation.py',
    property_files=[evaluation_report],
)

#### Register model in Model Registry step

Once we've evaluated the model's peformance, we'll want to register the model in a Model Registry.

In [ ]:
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri='{}/evaluation.json'.format(
            evaluation_step.arguments['ProcessingOutputConfig']['Outputs'][0]['S3Output'][
                'S3Uri'
            ]
        ),
        content_type='application/json',
    )
)

model = Model(
    image_uri=estimator.training_image_uri(),
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    entry_point=estimator.entry_point,
    role=role_arn,
    sagemaker_session=session
)

model_registry_args = model.register(
    content_types=['text/csv'],
    response_types=['application/json'],
    inference_instances=['ml.t2.medium', 'ml.m5.xlarge'],
    transform_instances=['ml.m5.xlarge'],
    model_package_group_name=model_package_group_name,
    approval_status='PendingManualApproval',
    model_metrics=model_metrics
)

register_step = ModelStep(
    name='RegisterModel',
    step_args=model_registry_args
)

But we'll only want to register the model if its performance meets a predefined threshold that we set. So let's create a Condition Step that says if our model's MSE values is less than 80000000.0, then we'll registery the model.

In [ ]:
# Condition step for evaluating model quality and branching execution

cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=evaluation_step.name,
        property_file=evaluation_report,
        json_path='regression_metrics.mse.value',
    ),
    right=80000000.0,
)
condition_step = ConditionStep(
    name='CheckEvaluation',
    conditions=[cond_lte],
    if_steps=[register_step],
    else_steps=[],
)

#### Assemble the training pipeline

Though easier to reason with, the parameters and steps don't need to be in order. The pipeline DAG will parse it out properly.

In [ ]:
# pipeline_name = 'synthetic-housing-training-pipeline-{}'.format(strftime('%d-%H-%M-%S', gmtime()))
pipeline_name = 'synthetic-housing-training-pipeline'
step_list = [preprocess_dataset_step,
             training_step,
             evaluation_step,
             condition_step]

training_pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        processing_instance_type
    ],
    steps=step_list
)

# Note: If an existing pipeline has the same name it will be overwritten.
training_pipeline.upsert(role_arn=role_arn)

# Viewing the pipeline definition will all the string variables interpolated may help debug pipeline bugs. It is commented out here due to length.
#json.loads(training_pipeline.definition())

#### Execute the training pipeline

In [ ]:
execution = training_pipeline.start(
    parameters = {
        'ProcessingInstanceType': 'ml.m5.large'
    }
)

Check on status of pipeline

In [ ]:
execution.describe()

In [ ]:
execution.list_steps()

In [ ]:
execution.wait()

## STAGE 2 -Deployment pipeline with SageMaker Pipelines

Now let's create a separate pipeline that will take the model that was registered in Model Registry and deploy it as a SageMaker hosted endpoint.

First we'll specify the input parameters to our deployment pipeline so that we can reuse it.

In [ ]:
%%writefile ./pipeline_scripts/lambda_deploy.py
'''
This Lambda function creates an Endpoint Configuration and deploys a model to an Endpoint. 
The name of the model to deploy is provided via the `event` argument
'''

import json
import boto3
import traceback
import time

region = boto3.Session().region_name
sm_client = boto3.client('sagemaker', region_name=region)

print(f'boto3 version: {boto3.__version__}')
print(f'region: {region}')


def lambda_handler(event, context):
    """
    Approve the model package for deployment, create a SageMaker model
    """
    
    region = event['region']
    aws_account_id = event['aws_account_id']
    model_package_group_name = event['model_package_group_name']
    instance_count = event['instance_count']
    role_arn = event['role_arn']
    
    # Optional fields
    try:
        model_package_version = event['model_package_version']
    except:
        # Get the latest version
        model_package_version = sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)['ModelPackageSummaryList'][0]['ModelPackageVersion']
        
    try:
        model_name = event['model_name']
    except:
        model_name = f'{model_package_group_name}-model-{model_package_version}'
    
    model_package_version_arn = f'arn:aws:sagemaker:{region}:{aws_account_id}:model-package/{model_package_group_name}/{model_package_version}'
    print(f'Using model package version ARN: {model_package_version_arn}')
    model_package_details = sm_client.describe_model_package(ModelPackageName=model_package_version_arn)

    realtime_inference_instance_types = model_package_details['InferenceSpecification']['SupportedRealtimeInferenceInstanceTypes']
    
    container_list = [{'ModelPackageName': model_package_version_arn}]
    
    # Approve model package to be used as SageMaker model and for deployment
    sm_client.update_model_package(ModelPackageArn=model_package_version_arn,
                                   ModelApprovalStatus='Approved')
    
    endpoint_name = f'{model_name}-endpoint'
    print(f"Lambda:Model:NAME:PASSED:{model_name}:")
    
    # Delete in case this pipeline runs again
    #  We have to keep the model name the same since we do not return the Model name here
    try:
        sm_client.delete_model(ModelName=model_name)
        print(f"Lambda:Model:{model_name}::DELETED::")
    except:
        print(f"Lambda:IGNORE::ERROR:Delete:model:Probably First RUN:{traceback.format_exc()}:")
    try:
        sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
        print(f"Lambda:EndPointConfig:{endpoint_name}::DELETED::")
    except:
        print(f"Lambda:IGNORE::ERROR:Delete:EndpointConfig:Probably First RUN:{traceback.format_exc()}:")
    try:
        sm_client.delete_endpoint(EndpointName=endpoint_name)
        print(f"Lambda:EndPoint:{endpoint_name}::DELETED:")
        
        time.sleep(60) # sleep for 60 seconds to ensure the model end point is indeed deleted if it was there
    except:
        print(f"Lambda:IGNORE::ERROR:Delete:EndPoint:Probably First RUN:{traceback.format_exc()}:")
        
    print(f"Lambda:Creating:endpoint:{endpoint_name}")
    sm_client.create_model(ModelName=model_name,
                           Containers=container_list,
                           ExecutionRoleArn=role_arn)

    
    create_endpoint_config_response = sm_client.create_endpoint_config(
        EndpointConfigName=endpoint_name,
        ProductionVariants=[
            {
                'InstanceType': realtime_inference_instance_types[0],
                'InitialVariantWeight': 1,
                'InitialInstanceCount': instance_count,
                'ModelName': model_name,
                'VariantName': 'AllTraffic',
            }
        ],
    )

    create_endpoint_response = sm_client.create_endpoint(EndpointName=endpoint_name, EndpointConfigName=endpoint_name)
    print(f"Lambda:end:point:create_endpoint_response={create_endpoint_response}:")
    time.sleep(60) # sleep for 60 seconds to ensure the end point is created
    
    return {
        'statusCode': 200,
        #'body': json.dumps('Created Endpoint!')
        'body': json.dumps(create_endpoint_response)
    }


#### Parametrize the Model Name

In [ ]:
model_name = ParameterString(
    name='ModelName',
    default_value='my-awesome-model'
)

Next, we'll create a Lambda function that will pull the specified model (or latest model) from the Model Registry and deploy as a Sagemaker endpoint.

In [ ]:
lambda_name = 'sagemaker-pipelines-deploy-model'
lambda_role = "arn:aws:iam::622343165275:role/sagemaker-lambda-exec"
import traceback
lambda_function = Lambda(
    function_name=lambda_name,
    execution_role_arn=lambda_role,
    script='./pipeline_scripts/lambda_deploy.py',
    handler='lambda_deploy.lambda_handler',
    timeout=600,
    memory_size=3000,
)
try:
    lambda_function.delete()
except:
    print('Lambda function Does Not exits:First run: IGNORE Error')
    print(traceback.format_exc())
try:
    lambda_function_response = lambda_function.create()
    lambda_function_arn = lambda_function_response['FunctionArn']
    print(f'Lambda function arn: {lambda_function_arn}')
except:
    print('Lambda function already exists!')
    print(traceback.format_exc())

Now we'll create a Lambda step for our pipeline and associate it with the new Lambda function we just created.

In [ ]:
# The dictionary retured by the Lambda function is captured by LambdaOutput, each key in the dictionary corresponds to a
# LambdaOutput

output_param_1 = LambdaOutput(output_name='statusCode', output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name='body', output_type=LambdaOutputTypeEnum.String)

deploy_lambda_step = LambdaStep(
    name='LambdaStepDeploy',
    lambda_func=lambda_function,
    inputs={
        'region': region,
        'aws_account_id': aws_account_id,
        'model_package_group_name': model_package_group_name,
        'model_name': model_name,
        'instance_count': 1,
        'role_arn': role_arn
    },
    outputs=[
        output_param_1, 
        output_param_2
    ],
)

Excellent, now we just need to assemble the pipeline.

#### Assemble the deployment pipeline

In [ ]:
# pipeline_name = 'synthetic-housing-deployment-pipeline-{}'.format(strftime('%d-%H-%M-%S', gmtime()))
pipeline_name = 'synthetic-housing-deployment-pipeline'
step_list = [deploy_lambda_step]

deployment_pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        model_name
    ],
    steps=step_list
)

# Note: If an existing pipeline has the same name it will be overwritten.
deployment_pipeline.upsert(role_arn=role_arn)

# Viewing the pipeline definition will all the string variables interpolated may help debug pipeline bugs. It is commented out here due to length.
json.loads(deployment_pipeline.definition())

#### Execute the deployment pipeline

In [ ]:
deployed_model_name = 'my-xgboost-model'
lambda_execution = deployment_pipeline.start(
    parameters = {
        'ModelName': deployed_model_name
    }
)

Check on status of pipeline

In [ ]:
lambda_execution.describe()

In [ ]:
lambda_execution.list_steps()

In [ ]:
lambda_execution.wait()

### Wait for the End point to be in service

In [ ]:

resp = sagemaker_client.describe_endpoint(EndpointName=f'{deployed_model_name}-endpoint')
status = resp["EndpointStatus"]
print("Endpoint:Status: " + status)

sagemaker_client.get_waiter("endpoint_in_service").wait(EndpointName=f'{deployed_model_name}-endpoint')

resp = sagemaker_client.describe_endpoint(EndpointName=f'{deployed_model_name}-endpoint')
status = resp["EndpointStatus"]
print("EndPoint:Arn: " + resp["EndpointArn"])
print("EndPoint:Status: " + status)

if status != "InService":
    raise Exception("Endpoint creation did not succeed")

#### Test the SageMaker endpoint

Let's now send some data to the endpoint and test it is working properly.

For this, we first load our test data from Feature Store

In [ ]:
test_data_path = "./data/test/test.csv"
# read data input
df = pd.read_csv(test_data_path)
df.head()

In [ ]:
df.columns

Then we query the endpoint once it is available

In [ ]:
import time
response_status = 'None'
while response_status != 'InService':
    if response_status != 'None':
        time.sleep(120) # wait until endpoint is in service
    response = sagemaker_client.describe_endpoint(EndpointName=f'{deployed_model_name}-endpoint')
    response_status = response['EndpointStatus']
# Attach to the SageMaker endpoint
predictor = Predictor(endpoint_name=f'{deployed_model_name}-endpoint',
                      sagemaker_session=session,
                      serializer=CSVSerializer(),
                      deserializer=CSVDeserializer())

# Get a real-time prediction
predictor.predict(df.drop(columns=["PRICE"]).to_csv(index=False, header=False))[0]